In [1]:
# Add script to include project root in sys.path

import sys
from pathlib import Path

# Add project root regardless of launch location
start_path = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
project_root = next((p for p in [start_path, *start_path.parents] if (p / ".git").exists()), start_path)
sys.path.append(str(project_root))


In [2]:
# Import necessary libraries

import os
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

from src.gradcam import GradCAM


In [3]:
# Define constants and configurations

DATA_DIR = project_root / "static/output"
MODEL_PATH = project_root / "saved_models/resnet50_best.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [4]:
# Load the pre-trained model

model = models.resnet50(weights=None)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)

state_dict = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state_dict, strict=False)
model = model.eval().to(device)

print("Model loaded and ready on:", device)


Model loaded and ready on: cpu


In [5]:
# Define utility functions: Load image function (PNG, JPG, DICOM)

def load_image(path: Path):
    ext = path.suffix.lower()
    if ext in [".png", ".jpg", ".jpeg"]:
        return Image.open(path).convert("RGB")

    if ext == ".dcm":
        from pydicom import dcmread
        ds = dcmread(str(path))
        img = ds.pixel_array
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        img = cv2.cvtColor(img.astype("uint8"), cv2.COLOR_GRAY2RGB)
        return Image.fromarray(img)

    raise ValueError(f"Unsupported format: {ext}")


In [6]:
# Define functions to compute image properties (Brightness, Resolution, Confidence)

def compute_brightness(img: np.ndarray):
    return img.mean()

def compute_resolution(img: np.ndarray):
    return img.shape[0] * img.shape[1]

def predict(img_tensor):
    with torch.no_grad():
        outputs = model(img_tensor)
        probs = F.softmax(outputs, dim=1)
    return probs[0,0].item(), probs[0,1].item()  # normal, pneumonia
